In [ ]:
!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.5/107.7 GB disk)


In [ ]:

from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
model = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:17<00:00, 2.93MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/datasetcse463.zip -d /content/cse463project/



Streaming output truncated to the last 5000 lines.
 extracting: /content/cse463project/train/labels/589_jpg.rf.bd6b33373ba44dbe0600026897b68393.txt  
 extracting: /content/cse463project/train/labels/58_jpg.rf.4be091701dba757220a598817204aa4c.txt  
 extracting: /content/cse463project/train/labels/5aCb5hhRM5uhRk9h7X3J_original_jpg.rf.c6067373ab411eef46b11c9942282df2.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.0b8b2eb26fd78e1b6ac1fbd8e07578c9.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.2cfa5c8ea60d12bf104292b894765d28.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.934cb1dff87a33b6166c65928500b905.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.b84067c310ce5d9a875447c5287b232f.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.cf9380364b8a17f99cc437982da2062f.txt  
 extracting: /content/cse463project/train/labels/5e1c1d79-73_jpg.rf.4fd43d012c741c6ae49a13ada8e60

In [ ]:
%cd /content


/content


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/cse463project/data.yaml epochs=5 imgsz=500 plots=True


New https://pypi.org/project/ultralytics/8.3.58 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/cse463project/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=500, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sav

In [ ]:
import gc
gc.collect()


113

In [ ]:
from IPython.display import Image
%cd /content
!find /content -name "confusion_matrix.png"
Image(filename='/content/runs/detect/train/confusion_matrix.png', width=600)



/content


In [ ]:
%cd /content
Image(filename=f'/content/runs/detect/train/results.png', width=600)

In [ ]:
%cd /content
Image(filename=f'/content/runs/detect/train/val_batch0_pred.jpg', width=600)

In [ ]:
%cd /content

!yolo task=detect mode=val model=/contentruns/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
%cd /content
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

# Define the base path where the folders are located
base_path = '/content/runs/detect/'

# List all directories that start with 'predict' in the base path
subfolders = [os.path.join(base_path, d) for d in os.listdir(base_path)
              if os.path.isdir(os.path.join(base_path, d)) and d.startswith('predict')]

# Find the latest folder by modification time
latest_folder = max(subfolders, key=os.path.getmtime)

image_paths = glob.glob(f'{latest_folder}/*.jpg')[:3]

# Display each image
for image_path in image_paths:
    display(Image(filename=image_path, width=600))
    print("\n")

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train/")

In [ ]:
# Run inference on your model on a persistent, auto-scaling, cloud API

# Load model
model = project.version(dataset.version).model
assert model, "Model deployment is still loading"

# Choose a random test image
import os, random
test_set_loc = dataset.location + "/test/images/"
random_test_image = random.choice(os.listdir(test_set_loc))
print("running inference on " + random_test_image)

pred = model.predict(test_set_loc + random_test_image, confidence=40, overlap=30).json()
pred

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# Mount Google Drive (if required)
from google.colab import drive
drive.mount('/content/drive')

# Unzip your dataset
!unzip /content/drive/MyDrive/datasetcse463.zip -d /content/cse463project/

# Paths
train_dir = "/content/cse463project/train"
val_dir = "/content/cse463project/val"

# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# Load Pretrained VGG16 without the top layer
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation="softmax")(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

# Summary of the model
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Save the model
model.save("/content/drive/MyDrive/vgg16_model.h5")

# Visualize Training Results
import matplotlib.pyplot as plt

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')


Streaming output truncated to the last 5000 lines.
 extracting: /content/cse463project/train/labels/58_jpg.rf.4be091701dba757220a598817204aa4c.txt  
 extracting: /content/cse463project/train/labels/5aCb5hhRM5uhRk9h7X3J_original_jpg.rf.c6067373ab411eef46b11c9942282df2.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.0b8b2eb26fd78e1b6ac1fbd8e07578c9.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.2cfa5c8ea60d12bf104292b894765d28.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.934cb1dff87a33b6166c65928500b905.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.b84067c310ce5d9a875447c5287b232f.txt  
 extracting: /content/cse463project/train/labels/5ccb80c7-8_jpg.rf.cf9380364b8a17f99cc437982da2062f.txt  
 extracting: /content/cse463project/train/labels/5e1c1d79-73_jpg.rf.4fd43d012c741c6ae49a13ada8e60cd1.txt  
 extracting: /content/cse463project/train/labels/5e1c1d79-73_jpg.rf.5dbbcb8802618630d77cd

FileNotFoundError: [Errno 2] No such file or directory: '/content/cse463project/val'